In [1]:
import json
import numpy as np
import pandas as pd
import sys, os
#!/usr/bin/python
# parent = os.path.dirname(os.path.realpath(__file__))
sys.path.append('~/mitie')

from mitie import *
from collections import defaultdict

ner = named_entity_extractor('../../../mitie/mitie-v0.2-python-2.7-windows-or-linux64/MITIE-models/english/ner_model.dat')

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans
from gensim import matutils
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.tokenize import word_tokenize


/home/ec2-user/anaconda2/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


### Open and read file

In [3]:
!ls

161102_times.json			 kmeans_test_4.csv
161103_times.json			 kmeans_test.csv
161105e_nyt.json			 Kojak_testing.ipynb
161107e_nyt.json			 news
161109_nytm2.json			 nytimes_extended.csv
161109_nytm.json			 nytimes_test_vocab.pkl
161110_nytm.json			 nyt nlp final.ipynb
161111_nytm.json			 nyt nlp final-multiday-Copy1.ipynb
161112_nytm.json			 nyt nlp final-multiday.ipynb
161114_nytm.json			 parse_nyt_data-Copy1.ipynb
161116_nytimes_extended_clusters_50.csv  parse_nyt_data.ipynb
161116_nytimes_extended_clusters.csv	 scrapy.cfg
161121nyt2.json				 tfidf_docsimilarity.ipynb
161121nyt.json				 ward_clusters.png
Clusters.png				 withspec2.csv
kmeans_test_2.csv			 withspec.csv
kmeans_test_3.csv


In [4]:
files = ['161102_times.json','161103_times.json','161105e_nyt.json','161105e_nyt.json','161107e_nyt.json','161109_nytm2.json','161110_nytm.json','161114_nytm.json']
splits = []
for file_ in files:
    f = open(file_,'r')
    filestring = f.read()
    split = filestring.replace(']\n','').replace('[\n','').split('\n')
    splits.extend(split)
final_dict = {}

In [5]:
print len(splits)

40472


### Preprocess text

In [6]:
for i in splits:
    try:
        temp = json.loads(i[:-1])
    except:
        continue   
     
    key = temp.keys()[0].split('?')[0]
    if '?' in key:
        print key
        break
    val = x.encode('utf-8').replace("\xe2\x80\x99","'").replace("\xe2\x80\x9c",'"').replace("\xe2\x80\x9d",'"').replace('\xe2\x80\x94',"'").replace('\xe2\x80\xa6','...')
       
    if val != '':
        final_dict[key] = val
    else:
        continue

In [7]:
final_dict.keys()[:10]

[u'http://www.nytimes.com/2014/01/10/world/middleeast/syrian-groups-try-to-recruit-us-travelers.html',
 u'http://www.nytimes.com/2014/03/19/sports/baseball/a-fast-growing-fraternity-in-the-game-tommy-john-patients.html',
 u'http://www.nytimes.com/2009/07/04/us/politics/04alaska.html',
 u'http://www.nytimes.com/2015/08/01/business/into-the-family-business-at-perdue.html',
 u'http://www.nytimes.com/2010/03/08/world/middleeast/08sunnis.html',
 u'http://www.nytimes.com/2009/06/17/world/middleeast/17tehran.html',
 u'http://www.nytimes.com/2015/04/25/opinion/laurent-fabius-our-climate-imperatives.html',
 u'http://www.nytimes.com/2015/02/17/opinion/yanis-varoufakis-no-time-for-games-in-europe.html',
 u'http://www.nytimes.com/2014/02/11/nyregion/de-blasio-state-of-the-city-speech.html',
 u'http://www.nytimes.com/2015/07/10/us/politics/jeb-bush-races-past-rivals-in-fund-raising-aided-by-super-pac-cash.html']

### Create dataframe

In [8]:
dump_dict = json.dumps(final_dict)
df = pd.read_json(dump_dict, typ = 'series')
df = pd.DataFrame(df).reset_index()
df.columns  = [['url','text']]
df['date'] = df['url'].apply(lambda x: x.split('.com/')[1].split('/')[:3])
df['section'] = df['url'].apply(lambda x: x.split('.com/')[1].split('/')[3])
df['length'] = df['text'].apply(lambda x: len(x))

In [9]:
df['length'].sum()

155974508

### Named Entitiy Recognition

In [10]:
cv2 = CountVectorizer(stop_words='english',max_df = 0.2, min_df = 0.1, strip_accents = 'unicode',analyzer='word').fit(df['text'])

KeyboardInterrupt: 

In [11]:
def get_entities(x, cv=cv2):
    x = tokenize(x)
    entities = ner.extract_entities(x)
    entities_list = []
    common_words = cv.vocabulary_.keys()
    for e in entities:
        rangee = e[0]
        tag = e[1]
        score = e[2]
        score_text = "{:0.3f}".format(score)
        entity_text = " ".join(x[i] for i in rangee)
        entities_list.append(entity_text)
    for i in x:
        if i.lower() in common_words:
            entities_list.append(i)
    return entities_list
        

NameError: name 'cv2' is not defined

In [12]:
def get_entities(x):
    x = tokenize(x)
    entities = ner.extract_entities(x)
    entities_list = []
    for e in entities:
        rangee = e[0]
        tag = e[1]
        score = e[2]
        score_text = "{:0.3f}".format(score)
        entity_text = " ".join(x[i] for i in rangee)
        entities_list.append(entity_text)
    return entities_list

In [14]:
get_entities(df['text'][0])

['HONG KONG',
 'Bishop Francis X. Ford',
 'Brooklyn-born',
 'Meihsien',
 'Kwangtung Province',
 'Canton',
 'Chinese',
 'Communist']

#### Lemmatization for Count Vectorizer

In [ ]:
lemmatizer = WordNetLemmatizer()
def lemmatize_me(x):
    x = word_tokenize(x)
    x_tagged = nltk.pos_tag(x)
    return_list = []
    for i in x_tagged:
        if 'V' in i[1]:
            value = 'v'
        else:
            value = 'n'
        return_list.append(lemmatizer.lemmatize(i[0], pos = value))
    return return_list

#### create countvectorizer object, transform text to sparse matrix using named entitiy processor

In [ ]:
cv = CountVectorizer(analyzer = get_entities,stop_words='english', max_df = 0.8, min_df = 0.01, strip_accents='unicode', ngram_range=(1,3)).fit(df['text'])

In [ ]:
sm = cv2.transform(df['text'])

### Fit TF IDF and transform sparse matrix from count vectorizer

In [ ]:
tt = TfidfTransformer().fit_transform(sm)


In [ ]:
tt

In [ ]:
tv = TfidfVectorizer(analyzer= get_entities, stop_words='english', max_df=0.20, min_df=0.005, strip_accents='unicode').fit(df['text'])


In [ ]:
tfidf = tv.transform(df['text'])

In [ ]:
tfidf

#### Latent Semantic Analysis

##### Trained on count vectorizer sparse matrix, better explained variance ratio

In [ ]:
lsa = TruncatedSVD(n_components=20)
nyt_lsa = lsa.fit_transform(sm)
nyt_lsa = Normalizer(copy=False).fit_transform(nyt_lsa)

In [ ]:
nyt_lsa

In [ ]:
lsa.explained_variance_

#### Let's do the damn thing. Kmeans 

In [ ]:
kmeans = KMeans(n_clusters=7)
km = kmeans.fit(tt)
clusters = km.labels_.tolist()

In [ ]:
df['clusters'] = clusters
df['clusters2'] = clusters2

In [ ]:
df[['url','clusters','clusters2','spec']].to_csv('withspec2.csv')

In [ ]:
kmeans2 = KMeans(n_clusters=50)
km2 = kmeans2.fit(tfidf)
clusters2 = km2.labels_.tolist()

#### Spectral wtf is this shit?

In [ ]:
import gc
gc.collect()

In [ ]:
from sklearn.cluster import SpectralClustering

In [ ]:
spec = SpectralClustering(n_clusters=25, affinity='cosine', n_init=50)

In [ ]:
spec_fit = spec.fit(tfidf)
spec_pred = spec.fit_predict(tfidf)
df['spec'] = spec_pred

In [ ]:
af_mat = pd.DataFrame(spec_fit.affinity_matrix_)

In [ ]:
af_mat

In [ ]:
pd.options.display.max_colwidth = 250

In [ ]:
print df[['url']][df['spec'] ==0]

In [ ]:
# spec = SpectralClustering(n_clusters=50, eigen_solver='arpack', affinity='sigmoid') #good clustering for certain topics mosul(7??) comey, clinton, trump, india 12, 

In [ ]:
spec_fit.n_neighbors

In [ ]:
"""
Visualization of matrix block structure
(eg, pairwise similarity or co-occurrence)
Requires scaledimage.py for intensity plots
David Andrzejewski
"""
import numpy as NP
import matplotlib.pyplot as P
import matplotlib.lines as L
import sklearn.cluster as SKLC

# from scaledimage import scaledimage

def blockviz(affinity, nclust, ax=None):
    """ 
    Visualize block-structure of affinity matrix 
    affinity = NxN non-negative affinity matrix
    nclust = number of clusters to use
    ax = matplotlib Axes to draw on 
    Rely on caller to .show()
    """
    # Activate the appropriate Axes in pyplot
    if(ax == None):
        ax = P.figure().gca()
    # Do spectral clustering
    ndata = affinity.shape[0]
    c = SKLC.SpectralClustering(n_clusters=nclust, affinity='linear', random_state=1445)
    c.fit(affinity)
    # Extract cluster labels and sort indices to align with clusters
    sortidx = []
    for ki in range(nclust):
        sortidx += getlabeled(c.labels_, ki)
    sorted_affinity = affinity.copy()
    sorted_affinity = sorted_affinity[sortidx,:]
    sorted_affinity = sorted_affinity[:,sortidx]
    # Intensity plot of affinity
    width = 10
    scaledimage(sorted_affinity,
                pixwidth=width, grayscale=True, ax=ax)
    # Draw recovered cluster boundaries
    kstart = 0
    clus_size = []
    clus_start = []
    for ki in range(nclust):    
        clustki = getlabeled(c.labels_, ki)
        clus_size.append(clustki)
        kstart += len(clustki)
        clus_start.append(kstart)
    return c.labels_

def logistic(val):
    """ Logistic function """
    return float(1) / (1 + NP.exp(-1 * val))

def getlabeled(labels, ki):
    """ Get indices where labels==ki """
    return [idx for (idx, val) in 
            enumerate(labels) if val == ki]

def drawH(ax, y, xstart, xend):
    """ Draw horiztonal line """
    ax.add_line(L.Line2D([xstart, xend], 
                  [y, y],
                  color='r'))

def drawV(ax, x, ystart, yend):
    """ Draw vertical line """
    ax.add_line(L.Line2D([x, x],
                         [ystart, yend], 
                         color='r'))

def drawClust(ax, kstart, kend, kmax, scale=1):
    """ Draw bounding box for cluster """
    skstart = scale * kstart
    skend = scale * kend
    skmax = scale * kmax
    if(skstart == 0):
        # Upper-left cluster: only draw bottom-right borders
        drawH(ax, skmax-skend, skstart, skend)
        drawV(ax, skend, skmax-skstart, skmax-skend)
    elif(skend == skmax):
        # Lower-right cluster: only draw top-left borders
        drawH(ax, skmax-skstart, skstart, skend)
        drawV(ax, skstart, skmax-skstart, skmax-skend)
    else:
        # Otherwise, draw all 4 borders
        drawH(ax, skmax-skend, skstart, skend)
        drawV(ax, skend, skmax-skstart, skmax-skend)
        drawH(ax, skmax-skstart, skstart, skend)
        drawV(ax, skstart, skmax-skstart, skmax-skend)
        
import numpy as NP
import matplotlib.pyplot as P
import matplotlib.ticker as MT
import matplotlib.cm as CM
 
def scaledimage(W, pixwidth=20, ax=None, grayscale=True):
    """
    Do intensity plot, similar to MATLAB imagesc()
 
    W = intensity matrix to visualize
    pixwidth = size of each W element
    ax = matplotlib Axes to draw on 
    grayscale = use grayscale color map
 
    Rely on caller to .show()
    """
    # N = rows, M = column
    (N, M) = W.shape 
    # Need to create a new Axes?
    if(ax == None):
        ax = P.figure(figsize=(20,20)).gca()
    # extents = Left Right Bottom Top
    exts = (0, pixwidth * M, 0, pixwidth * N)
    if(grayscale):
        ax.imshow(W,
                  interpolation='nearest',
                  cmap=CM.gray,
                  extent=exts)
    else:
        ax.imshow(W,
                  interpolation='nearest',
                  extent=exts)
 
    ax.xaxis.set_major_locator(MT.NullLocator())
    ax.yaxis.set_major_locator(MT.NullLocator())
    return ax
 
if __name__ == '__main__':
    # Define a synthetic test dataset
     testweights = NP.array([[0.25, 0.50, 0.25, 0.00],
                            [0.00, 0.50, 0.00, 0.00],
                            [0.00, 0.10, 0.10, 0.00],
                            [0.00, 0.00, 0.25, 0.75]])
    # Display it
#     ax = scaledimage(testweights)
#     P.show()


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

####  cosine, 25 clusters

In [ ]:
blockviz(np.matrix(af_mat), 25, ax=P.figure(figsize=(50,50)).gca())
P.savefig('Clusters.png', dpi = 300)

#### cosine 30^

In [ ]:
blockviz(np.matrix(af_mat), 20)

#### cosine 20 ^

In [ ]:
blockviz(np.matrix(af_mat), 25, ax=P.figure(figsize=(100,100)).gca())


#### cosine 25

In [ ]:
blockviz(np.matrix(af_mat), 20)

#### poly 50 ^

In [ ]:
blockviz(np.matrix(af_mat), 20)

#### poly 30

In [ ]:
blockviz(np.matrix(af_mat), 20)

#### Poly 20

In [ ]:
import seaborn as sns